In [1]:
import requests
import json
import pandas as pd
import time
import convex as cx
import tmqa2 as tmqa2

Autosaving every 180 seconds





Using TensorFlow backend.


Loading the params file
Input encoding {'o': 2, '{': 3, '.': 4, 'J': 5, '0': 6, '1': 7, '<': 8, 'B': 9, 'd': 10, '£': 11, 'e': 12, '6': 13, '!': 14, 'O': 15, 'M': 16, 'X': 17, 'f': 18, 't': 19, 'C': 20, 'V': 21, 'z': 22, 'K': 23, '\\': 24, '9': 25, 'P': 26, 'S': 27, '/': 28, '₹': 29, 'F': 30, 'G': 31, '=': 32, '8': 33, ')': 34, '+': 35, ']': 36, 'U': 37, "'": 38, '"': 39, 'g': 40, 'N': 41, 'r': 42, 'u': 43, '&': 44, '$': 45, 'x': 46, '%': 47, ':': 48, '@': 49, '^': 50, 'I': 51, 'L': 52, 'Z': 53, 'h': 54, 'W': 55, 'A': 56, 'v': 57, '?': 58, '2': 59, '~': 60, 's': 61, 'T': 62, 'R': 63, ',': 64, '|': 65, '4': 66, '>': 67, 'y': 68, '(': 69, '[': 70, 'k': 71, 'H': 72, 'l': 73, 'j': 74, '7': 75, 'n': 76, 'i': 77, 'D': 78, 'Q': 79, ' ': 80, 'm': 81, 'Y': 82, '*': 83, '}': 84, '#': 85, 'p': 86, 'q': 87, '5': 88, 'c': 89, '`': 90, 'a': 91, 'b': 92, 'w': 93, '3': 94, 'E': 95, ';': 96, '-': 97}
Input decoding {2: 'o', 3: '{', 4: '.', 5: 'J', 6: '0', 7: '1', 8: '<', 9: 'B', 10: 'd', 11: '£', 12: '

In [2]:
conversations_path = "/data/users/romain.claret/tm/mse.tm.chatbot.base/data/convex/test_set/test_set_ALL.json"
with open(conversations_path, "r") as data:
    conversations = json.load(data)

In [3]:
def pickle_data(df, filename):
    pd.set_option('display.max_columns', None)
    filename = "/data/users/romain.claret/tm/mse.tm.chatbot.base/benchmark_pickles/"+filename+'.pickle.bz2'
    #df.summary = df.summary.map(sanitize_str)
    print("Saving Dataframe Done!",filename)
    return df.to_pickle(filename, compression='bz2')

In [4]:
#lang: en, fr, de, it, es, zh
#kb: dbpedia, wikidata, dblp, freebase
def ask_qanswer(question):
    data = {'query': question,'lang': 'en','kb': 'wikidata'}
    query = requests.post('http://qanswer-core1.univ-st-etienne.fr/api/gerbil', data=data)
    if not query:
        return False
    if (query.json()['questions'][0]['question']['answers']) == None:
        return False
    #if (query.json()['questions'][0]['question']['answers'].replace('\n', '')) == None:
    #    return False
    #print(query.json()['questions'][0]['question']['answers'].replace('\n', '').get("results"))
    try:
        response = (json.loads(query.json()
                .get("questions")[0]
                .get("question")
                .get("answers")
                .replace('\n', ''))
         .get("results").get("bindings"))
    except:
        return False
    
    if response:
        return response[0].get("o1").get("value")[len("http://www.wikidata.org/entity/"):] if response[0].get("o1") is not None else False
    else:
        return False

#ask_qanswer("Who is the wife of Barack Obama")
#ask_qanwser("Which equestrian was born in dublin?")
#ask_qanswer("what is the main language spoken in a ghentar si muore facile")
#ask_qanswer("was the film helpmates in color or black-and-white?")

In [5]:
def ask_platypus(question):
    headers = {'Accept': 'application/json','Accept-Language': 'en',}
    params = (('q', question),('lang', 'en'))

    response = requests.get('https://qa.askplatyp.us/v0/ask', headers=headers, params=params)
    if response:
        if type(response.json()['member']) is list:
            #print(response.json()['member'][0]['result'])
            if response.json()['member'] != []:
                if '@id' in (json.dumps(response.json()['member'][0]['result'])):
                    try:
                        ps_result = (json.dumps(response.json()['member'][0]['result']['@id']))
                    except:
                        return False
                else: return False
            else: return False
        else:
            try:
                if '@id' in (json.dumps(response.json()['member']['result'])):
                    ps_result = (json.dumps(response.json()["member"]['result']['@id']))
                else: return False
            except:
                return False
    else: return False
    ps_result = ps_result[4:-1]
    #print(result[:1])
    if ps_result[:1] != 'P' and ps_result[:1] != 'Q':
        return False
    return ps_result
#ask_platypus("Which genre of album is harder.....faster?")
#ask_platypus("how does engelbert zaschka identify")
#ask_platypus("Which Swiss conductor's cause of death is myoc...")
#ask_platypus("where was padraic mcguinness's place of death")
#ask_platypus("was the film helpmates in color or black-and-white?")
#ask_platypus("Who created the show life on earth")
#ask_platypus("Who is the wife of Barack Obama")

In [6]:
def ask_convex(question):
    cx_result = cx.answer_complete_question(question, cx.tagmeToken)['answers'][0]['answer']
    #print(cx_result)
    #answer = str(cx.wd.wikidata_id_to_label(result['answers'][0]['answer']))
    try:
        if not cx_result:
            return cx_result
        if cx_result[:1] != 'P' and cx_result[:1] != 'Q':
            return False
        return cx_result
    except:
        return False

#ask_convex("Which actor voiced the Unicorn in The Last Unicorn?")
#ask_convex("Which genre of album is harder.....faster?")
#ask_convex("Which label is somevelvetsidewalk signed to ttle of fort fisher ")
#ask_convex("Who is the wife of Barack Obama")
#ask_convex("100% senorita is a television show in what language?")

In [7]:
def ask_tmqa(question, verbose=False):
    tmqa_result = tmqa2.answer_question(question, verbose=verbose)
    #print(tmqa_result)
    if not tmqa_result:
        return tmqa_result
    if tmqa_result[0][0][:1] != 'P' and tmqa_result[0][0][:1] != 'Q':
        return False
    return tmqa_result[0]

#answer = ask_tmqa_1("Which actor voiced the Unicorn in The Last Unicorn?", verbose=True)
#answer = ask_tmqa_1("what's akbar tandjung's ethnicity", verbose=True)
#ask_tmqa_1("Which genre of album is harder.....faster?")
#ask_tmqa_1("Which label is somevelvetsidewalk signed to ttle of fort fisher ")
#ask_tmqa_1("Who is the wife of Barack Obama")
#print(answer)

In [11]:
HEADERS = ['question', 'answer', 'qanswer', 'platypus', 'convex',
           'tm', "tm_time", "tm_top2", "tm_top3", "tm_top4", "tm_top5", "tm_topall"]
df = pd.DataFrame(columns=HEADERS)

In [12]:
#for conversation in conversations:
#    questions = [turn['question'] for turn in conversation['questions']]
#    print(questions)
#    golden_answers = [tmqa2.wikidata_url_to_wikidata_id(turn['answer']) for turn in conversation['questions']]
#    print(golden_answers)
#    
#    break

In [13]:
# Evaluate

start_time = time.time()
conversations_len = len(conversations)

for i, conversation in enumerate(conversations):
    if i >= len(df):
        
        questions = [turn['question'] for turn in conversation['questions']]
        question = questions[0]
        answers = [tmqa2.wikidata_url_to_wikidata_id(turn['answer']) for turn in conversation['questions']]
        answer = answers[0]
        print("\r\t>>> Processing {}/{}: ({}) {}".format(i,conversations_len,answer,question), end='                                  ')
        
        print("processing: ask_qanswer")
        result_qanswer = ask_qanswer(question)
        print("processing: ask_platypus")
        result_platypus = ask_platypus(question)
        print("processing: ask_convex")
        result_convex = ask_convex(question)
        print("\n",i, "-> qAnswer", result_qanswer) if result_qanswer == answer else False
        print("\n",i, "-> Platypus", result_platypus) if result_platypus == answer else False
        print("\n",i, "-> Convex", result_convex) if result_convex == answer else False

        df_tm = False
        df_tm_top2 = False
        df_tm_top3 = False
        df_tm_top4 = False
        df_tm_top5 = False
        df_tm_topall = False
        
        print("processing: ask_tmqa")
        start_time_tmqa = time.time()
        result_tmqa = ask_tmqa(question, verbose=True)    
        if result_tmqa:
            df_tm = result_tmqa[0]
            if answer in result_tmqa[:2]:
                df_tm_top2 = True
            if answer in result_tmqa[:3]:
                df_tm_top3 = True
            if answer in result_tmqa[:4]:
                df_tm_top4 = True
            if answer in result_tmqa[:5]:
                df_tm_top5 = True
            if answer in result_tmqa:
                df_tm_topall = True
            
        df_tm1_time = round(time.time()-start_time_tmqa,2)
        
        df = df.append({"question":question, 'answer':answer, 'qanswer':result_qanswer, 'platypus':result_platypus, 'convex':result_convex,
                       "tm":df_tm, "tm_time":df_tm_time, "tm_top2":df_tm_top2, "tm_top3":df_tm_top3, "tm_top4":df_tm_top4, "tm_top5":df_tm_top5, "tm_topall":df_tm_topall},
                       ignore_index=True)

        print("\n",i, "-> tmqa", df_tm1) if str(df_tm) == str(answer) else False
        print("\n",i, "-> tmqa in answers", result_tmqa) if df_tm_topall == True else False
        
        print(df.tail(1))
        
        print("\n")
    
    #break

print("->\tRunning time is {}s".format(round(time.time()-start_time,2)))
df.tail(5)

NameError: name 'tmqa2' is not defined

In [9]:
len(df)

0

In [42]:
df.tail(10)

,question,answer,qanswer,platypus,convex,tm1,tm1_time,tm1_top2,tm1_top3,tm1_top4,tm1_top5,tm1_topall
0,What is the name of the writer of The Secret G...,Q276028,Q276028,False,False,Q443868,275.26,False,False,False,False,False
1,What network was Dexter on?,Q23589,Q23589,Q23577,Q23589,Q151885,236.90,False,False,False,False,False
2,When did the movie Grease come out?,1978-06-13T00:00:00Z,False,False,False,Q7752539,30.20,False,False,False,False,False
3,Who is the author that wrote the book Moby-Dick?,Q4985,False,False,Q4985,Q51159455,506.18,False,False,False,False,False
4,Who is the lead singer of 30 seconds to Mars?,Q484523,False,False,Q14179172,Q3012134,717.01,False,False,False,False,False
5,What was the first year Friends aired?,1994-01-01T00:00:00Z,False,Q387066,False,Q25292,355.74,False,False,False,False,False
6,In which year was the theatrical release of th...,2005-01-01T00:00:00Z,False,False,False,Q25292,562.36,False,False,False,False,False
7,Which character shares part of her name with t...,Q429828,False,False,Q1860,Q34770,1493.33,False,False,False,False,False
8,Which network originally broadcasted Lost?,Q169889,False,False,Q169889,Q131476,43.97,False,False,False,False,False
9,When was The Wonderful Wizard of Oz published?,1900-01-01T00:00:00Z,Q5541948,False,False,Q571,435.79,False,False,False,False,False


In [16]:
#SAVING
pickle_data(df_loaded, "benchmarking-qanswer-platypus-convex-tm1-from-0-to-"+str(len(df_loaded)))

Done! /data/users/romain.claret/tm/wikidata-simplequestions/benchmark_pickles/benchmarking-qanswer-platypus-convex-tm1-from-0-to-9961.pickle.bz2


In [4]:
#LOADING
df_loaded = pd.read_pickle("/data/users/romain.claret/tm/wikidata-simplequestions/benchmark_pickles/benchmarking-qanswer-platypus-convex-tm1-from-0-to-9961.pickle.bz2")

In [4]:
#df = df_loaded

In [5]:
df_loaded

,question,source,qanswer,platypus,convex,tm1,tm1_time,tm1_top2,tm1_top3,tm1_top4,tm1_top5,tm1_topall
0,Which genre of album is harder.....faster?,Q1096120,False,Q5487302,False,Q544440,3558.7,False,False,False,False,False
1,what city was alex golfis born in,Q1524,Q12751,False,Q83065,Q445559,723.63,False,False,False,False,False
2,what film is by the writer phil hay?,Q192979,False,False,Q1337306,False,1846.62,False,False,False,False,False
3,Where did roger marquis die,Q1637790,False,False,Q1637790,Q1637790,249.12,False,False,False,False,False
4,what was the cause of death of yves klein,Q12152,Q12152,False,Q12152,Q1394435,1710.05,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
9956,who was the creator of the fictional character...,Q311607,False,False,Q28975,False,False,False,False,False,False,False
9957,what's a college sporting event that took plac...,Q3994958,False,False,Q10219353,False,False,False,False,False,False,False
9958,what celestial object is 2974 holden,Q3863,False,False,False,False,False,False,False,False,False,False
9959,what is the film genre for snow falling on ced...,Q1257444,Q1054574,False,Q130232,False,False,False,False,False,False,False


In [65]:
#df_loaded = df_loaded.replace("", False)

In [ ]:
#df_loaded['qanswer'][34] = False

In [ ]:
#del df_loaded['tm2']

In [31]:
#df_loaded.rename({'mine':'tm1'}, axis=1, inplace=True)

In [39]:
#df_loaded['tm1_top4'] = "False"

In [43]:
#df_loaded = df_loaded[['question','source','qanswer','platypus','convex','tm1','tm1_time','tm1_top2','tm1_top3','tm1_top4','tm1_top5','tm1_topall']]

In [66]:
#df_loaded

,question,source,qanswer,platypus,convex,tm1,tm1_time,tm1_top2,tm1_top3,tm1_top4,tm1_top5,tm1_topall
0,Which genre of album is harder.....faster?,Q1096120,False,Q5487302,False,Q544440,3558.7,False,False,False,False,False
1,what city was alex golfis born in,Q1524,Q12751,False,Q83065,Q445559,723.63,False,False,False,False,False
2,what film is by the writer phil hay?,Q192979,False,False,Q1337306,False,1846.62,False,False,False,False,False
3,Where did roger marquis die,Q1637790,False,False,Q1637790,Q1637790,249.12,False,False,False,False,False
4,what was the cause of death of yves klein,Q12152,Q12152,False,Q12152,Q1394435,1710.05,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
9956,who was the creator of the fictional character...,Q311607,False,False,Q28975,False,False,False,False,False,False,False
9957,what's a college sporting event that took plac...,Q3994958,False,False,Q10219353,False,False,False,False,False,False,False
9958,what celestial object is 2974 holden,Q3863,False,False,False,False,False,False,False,False,False,False
9959,what is the film genre for snow falling on ced...,Q1257444,Q1054574,False,Q130232,False,False,False,False,False,False,False


In [17]:
print(df.dtypes)

question      object
source        object
qanswer       object
platypus      object
convex        object
tm1           object
tm1_time      object
tm1_top2      object
tm1_top3      object
tm1_top4      object
tm1_top5      object
tm1_topall    object
dtype: object


In [ ]:
df_loaded_len = len(df_loaded)
for i, question in enumerate(df_loaded['question']):
    if i >= 0:
    #if i >= 497:
        source = str(df_loaded['source'][i])
        print(str(i)+"/"+str(df_loaded_len),question,"-> source:",source)
        
        start_time = time.time()
        result_tmqa_1 = ask_tmqa_1(question, verbose=True)
        
        if result_tmqa_1:
            df_loaded['tm1'][i] = result_tmqa_1[0]
            if source in result_tmqa_1[:2]:
                df_loaded['tm1_top2'][i] = True
            if source in result_tmqa_1[:3]:
                df_loaded['tm1_top3'][i] = True
            if source in result_tmqa_1[:4]:
                df_loaded['tm1_top4'][i] = True
            if source in result_tmqa_1[:5]:
                df_loaded['tm1_top5'][i] = True
            if source in result_tmqa_1:
                df_loaded['tm1_topall'][i] = True
        else:
            df_loaded['tm1'][i] = False
        end_time = time.time()
        df_loaded['tm1_time'][i] = round(end_time-start_time,2)
        print("->\tRunning time is {}s".format(round(end_time-start_time,2)))
        print(str(str(df_loaded['tm1'][i])==str(source)),"---> result_tmqa_1:",str(result_tmqa_1)+"\n")

0/9961 Which genre of album is harder.....faster? -> source: Q1096120
-> q_nlp: Which genre of album is harder faster
-> q_themes: ([(album, ['Q1173065', 'Q10666342', 'Q482994']), (Genre, ['Q28490034', 'Q37446719', 'Q50913877', 'Q15758743']), (Album, ['Q10404408', 'Q11957254', 'Q6172625', 'Q11850638']), (genre, ['P136', 'Q188451', 'P415', 'Q483394'])], [Which genre])
-> q_themes_enhanced: []
-> q_predicates: [(is, ['P31', 'P433', 'P131'])]
-> q_focused_parts: [(genre, ['P136', 'Q188451', 'P415', 'Q483394']), (album, ['Q1173065', 'Q10666342', 'Q482994'])]
-> Building the graph with k_deep 50 ... (could be long)
-->  3860 nodes and 3854 edges
Stopping the computing here, too computational.
->	Running time is 55.14s
False ---> result_tmqa_1: False

1/9961 what city was alex golfis born in -> source: Q1524
-> q_nlp: what city was alex golfis born in
-> q_themes: ([(city, ['Q27676414', 'Q21583365', 'P131', 'Q15063611', 'Q27134986']), (City, ['Q1414771', 'Q1093831', 'Q1250599', 'Q1532871', '

---> Rebuilding the graph with k_deep 30 ... Previously: 741 nodes or 738 edges was above the limit...
---> Rebuilding the graph with k_deep 10 ... Previously: 741 nodes or 738 edges was above the limit...
-->  741 nodes and 738 edges
Stopping the computing here, too computational.
->	Running time is 174.6s
False ---> result_tmqa_1: False

5/9961 Which equestrian was born in dublin? -> source: Q6768445
-> q_nlp: Which equestrian was born in dublin
-> q_themes: ([(Dublin, ['Q1373580', 'Q36623636', 'Q16865980', 'Q1130963', 'Q173500', 'Q1761']), (Equestrian, ['Q5384297']), (equestrian, ['Q2730732'])], [Which equestrian, dublin])
-> q_themes_enhanced: []
-> q_predicates: [(was, ['P31', 'P641', 'P1196']), (born, ['P569', 'P19', 'P1477'])]
-> q_focused_parts: [(equestrian, ['Q2730732'])]
-> Building the graph with k_deep 50 ... (could be long)
-->  117 nodes and 112 edges
-> predicates_dict: {'P527': 1, 'P1465': 1, 'P20': 1, 'P1464': 1, 'P131': 3, 'P1013': 1, 'P19': 29, 'P735': 1, 'P425': 1,

-->  85 nodes and 82 edges
-> predicates_dict: {'P138': 1, 'P404': 2, 'P31': 8, 'P136': 1, 'P518': 2, 'P186': 2, 'P360': 1, 'P2283': 1, 'P131': 3, 'P1343': 1, 'P279': 3, 'P1535': 1, 'P421': 1, 'P400': 1, 'P1269': 2, 'P1419': 1, 'P361': 3, 'P641': 2, 'P170': 1, 'P123': 1, 'P195': 1, 'P17': 1, 'P366': 1}
-> paths_keywords: (['position', 'play', 'baseball', 'what', 'location'], {}, [what])
-> Computing possible paths... (could be long)
--> len(path_nodes): 2330
-> Filtering paths... (could be long)
--> len(paths_nodes_filtered): 448
-> Computing hypothesises...
--> hypothesises: [['Q11210', 0.7017946075373329], ['Q5369', 0.500689630073691], ['Q1041292', 0.3812286656005605], ['Q791187', 0.3809932368639528], ['Q368722', 0.3600417094308415], ['Q17334923', 0.32281853867140625], ['Q192388', 0.2664825605723096], ['Q27898614', 0.21526744433492415], ['Q20656446', 0.20830413106917348], ['Q1781513', 0.19955881456254704], ['Q12442118', 0.1089381020765273]]
-> Computing golden paths...
--> len(golden

-->  37 nodes and 32 edges
-> predicates_dict: {'P279': 2, 'P629': 1, 'P136': 1, 'P140': 1, 'P17': 1, 'P27': 1, 'P20': 1, 'P31': 5, 'P106': 2, 'P131': 1}
-> paths_keywords: ([], {}, [Who])
-> Computing possible paths... (could be long)
--> len(path_nodes): 0
-> Filtering paths... (could be long)
--> len(paths_nodes_filtered): 0
-> Computing hypothesises...
--> hypothesises: []
-> Looping on aggressive mode...
-> q_nlp: Who influenced michael mcdowell
-> q_themes: ([], [michael mcdowell, Influenced, Michael Mcdowell, influence michael mcdowell])
-> q_themes_enhanced: [('Michael', ['Q1286635', 'Q102162']), ('Mcdowell', ['Q6802793']), ('influence', ['Q18603649']), ('Influence', ['Q7741996', 'Q10857366'])]
-> q_predicates: [(influenced, ['P737']), (michael, ''), (mcdowell, '')]
-> q_focused_parts: []
-> Building the graph with k_deep 50 ... (could be long)
-->  51 nodes and 48 edges
-> predicates_dict: {'P734': 1, 'P735': 2, 'P279': 2, 'P629': 1, 'P136': 1, 'P131': 1, 'P140': 1, 'P27': 2, 

In [14]:
df.head(30)

,question,source,qanswer,platypus,convex,tm1,tm1_time,tm1_top2,tm1_top3,tm1_top4,tm1_top5,tm1_topall
0,Which genre of album is harder.....faster?,Q1096120,False,Q5487302,False,False,55.4,False,False,False,False,False
1,what city was alex golfis born in,Q1524,Q12751,False,Q83065,Q319608,222.1,False,False,False,False,False
2,what film is by the writer phil hay?,Q192979,False,False,Q1337306,Q134556,693.1,False,False,False,False,False
3,Where did roger marquis die,Q1637790,False,False,Q1637790,Q19970531,112.23,False,False,False,False,False
4,what was the cause of death of yves klein,Q12152,Q12152,False,Q12152,False,333.3,False,False,False,False,False
5,Which equestrian was born in dublin?,Q6768445,False,False,Q18339854,Q30,170.6,False,False,False,False,False
6,What is a tv action show?,Q5533291,False,False,Q238235,Q1914636,157.95,False,False,False,False,False
7,what's akbar tandjung's ethnicity,Q810714,False,False,Q6581097,Q6581097,74.86,False,False,False,False,False
8,What position does carlos gomez play?,Q5059480,Q164089,False,Q336286,Q11210,1376.8,False,False,False,False,False
9,how does engelbert zaschka identify,Q6581097,False,False,Q1779650,Q5,30.51,False,False,False,False,False


In [6]:
df_loaded.head(30)

,question,source,qanswer,platypus,convex,tm1,tm1_time,tm1_top2,tm1_top3,tm1_top4,tm1_top5,tm1_topall
0,Which genre of album is harder.....faster?,Q1096120,False,Q5487302,False,False,55.4,False,False,False,False,False
1,what city was alex golfis born in,Q1524,Q12751,False,Q83065,Q319608,222.1,False,False,False,False,False
2,what film is by the writer phil hay?,Q192979,False,False,Q1337306,Q134556,693.1,False,False,False,False,False
3,Where did roger marquis die,Q1637790,False,False,Q1637790,Q19970531,112.23,False,False,False,False,False
4,what was the cause of death of yves klein,Q12152,Q12152,False,Q12152,False,333.3,False,False,False,False,False
5,Which equestrian was born in dublin?,Q6768445,False,False,Q18339854,Q30,170.6,False,False,False,False,False
6,What is a tv action show?,Q5533291,False,False,Q238235,Q1914636,157.95,False,False,False,False,False
7,what's akbar tandjung's ethnicity,Q810714,False,False,Q6581097,Q6581097,74.86,False,False,False,False,False
8,What position does carlos gomez play?,Q5059480,Q164089,False,Q336286,Q11210,1376.8,False,False,False,False,False
9,how does engelbert zaschka identify,Q6581097,False,False,Q1779650,Q5,30.51,False,False,False,False,False


In [7]:
df = df_loaded.copy()
#df = df.replace("", False)

In [53]:
df_backup = df

,question,source,qanswer,platypus,convex,tm1,tm1_time,tm1_top2,tm1_top3,tm1_top4,tm1_top5,tm1_topall
0,Which genre of album is harder.....faster?,Q1096120,False,Q5487302,False,Q544440,3558.7,False,False,False,False,False
1,what city was alex golfis born in,Q1524,Q12751,False,Q83065,Q445559,723.63,False,False,False,False,False
2,what film is by the writer phil hay?,Q192979,False,False,Q1337306,False,1846.62,False,False,False,False,False
3,Where did roger marquis die,Q1637790,False,False,Q1637790,Q1637790,249.12,False,False,False,False,False
4,what was the cause of death of yves klein,Q12152,Q12152,False,Q12152,Q1394435,1710.05,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
9956,who was the creator of the fictional character...,Q311607,False,False,Q28975,False,False,False,False,False,False,False
9957,what's a college sporting event that took plac...,Q3994958,False,False,Q10219353,False,False,False,False,False,False,False
9958,what celestial object is 2974 holden,Q3863,False,False,False,False,False,False,False,False,False,False
9959,what is the film genre for snow falling on ced...,Q1257444,Q1054574,False,Q130232,False,False,False,False,False,False,False


In [8]:
max_row = 496
df_len = len(df)
df_qanswer_max = df[(df.index<=max_row) & (df.qanswer == df.source)]
df_qanswer_max_len = len(df_qanswer_max)

df_platypus_max = df[(df.index<=max_row) & (df.platypus == df.source)]
df_platypus_max_len = len(df_platypus_max)

df_convex_max = df[(df.index<=max_row) & (df.convex == df.source)]
df_convex_max_len = len(df_convex_max)

df_tm1_max = df[(df.index<=max_row) & (df.tm1 == df.source)]
df_tm1_max_len = len(df_tm1_max)

df_tm1_max_top2 = df[(df.index<=max_row) & (df.tm1_top2 == True)]
df_tm1_max_top2_len = len(df_tm1_max_top2)

df_tm1_max_top3 = df[(df.index<=max_row) & (df.tm1_top3 == True)]
df_tm1_max_top3_len = len(df_tm1_max_top3)

df_tm1_max_top4 = df[(df.index<=max_row) & (df.tm1_top4 == True)]
df_tm1_max_top4_len = len(df_tm1_max_top4)

df_tm1_max_top5 = df[(df.index<=max_row) & (df.tm1_top5 == True)]
df_tm1_max_top5_len = len(df_tm1_max_top5)

df_tm1_max_topall = df[(df.index<=max_row) & (df.tm1_topall == True)]
df_tm1_max_topall_len = len(df_tm1_max_topall)

print("qanswer:", df_qanswer_max_len,df_qanswer_max_len/max_row)
print("platypus:", df_platypus_max_len, df_platypus_max_len/max_row)
print("convex:", df_convex_max_len, df_convex_max_len/max_row)
print("tm1:", df_tm1_max_len, df_tm1_max_len/max_row)
print("tm1_top2:", df_tm1_max_top2_len, df_tm1_max_top2_len/max_row)
print("tm1_top3:", df_tm1_max_top3_len, df_tm1_max_top3_len/max_row)
print("tm1_top4:", df_tm1_max_top4_len, df_tm1_max_top4_len/max_row)
print("tm1_top5:", df_tm1_max_top5_len, df_tm1_max_top5_len/max_row)
print("tm1_topall:", df_tm1_max_topall_len, df_tm1_max_topall_len/max_row)

#df[ & (df.qanswer == df.source)]

qanswer: 77 0.15524193548387097
platypus: 5 0.010080645161290322
convex: 116 0.23387096774193547
tm1: 12 0.024193548387096774
tm1_top2: 19 0.038306451612903226
tm1_top3: 27 0.05443548387096774
tm1_top4: 30 0.06048387096774194
tm1_top5: 34 0.06854838709677419
tm1_topall: 46 0.09274193548387097


In [8]:
print("qanswer:",len(df[df.qanswer == df.source]),len(df[df.qanswer == df.source])/len(df))
print("platypus:",len(df[df.platypus == df.source]),len(df[df.platypus == df.source])/len(df))
print("convex:",len(df[df.convex == df.source]),len(df[df.convex == df.source])/len(df))
print("tm1:",len(df[df.tm1 == df.source]),len(df[df.tm1 == df.source])/len(df))
print("tm1_top2:",len(df[df.tm1_top2 == True]),len(df[df.tm1_top2 == True])/len(df))
print("tm1_top3:",len(df[df.tm1_top3 == True]),len(df[df.tm1_top3 == True])/len(df))
print("tm1_top4:",len(df[df.tm1_top4 == True]),len(df[df.tm1_top4 == True])/len(df))
print("tm1_top5:",len(df[df.tm1_top5 == True]),len(df[df.tm1_top5 == True])/len(df))
print("tm1_topall:",len(df[df.tm1_topall == True]),len(df[df.tm1_topall == True])/len(df))

qanswer: 1669 0.1675534584881036
platypus: 63 0.006324666198172874
convex: 2352 0.23612087139845397
tm1: 13 0.0013050898504166249
tm1_top2: 19 0.0019074390121473748
tm1_top3: 27 0.0027105712277883747
tm1_top4: 30 0.0030117458086537496
tm1_top5: 34 0.0034133119164742494
tm1_topall: 46 0.00461801023993575


In [62]:
df.tm1_top2 = df.tm1_top3